In [ ]:
#peak local max=> coordonnates
# k-distance
# Pour chaque point, on trie les distances vers tous les autres points 
# et on garde la distance vers le k-ième plus proche voisin (pas nécessairement le centroïde).
# Sert à capturer la densité locale autour d’un point.

In [ ]:
import os, sys
path_to_scr_folder=os.path.join(os.path.dirname(os.path.abspath('')), 'src')
sys.path.append(path_to_scr_folder)

In [ ]:
import read_chroma
from peak_detection import peak_detection
import numpy as np
import projection
import plot
import baseline_correction
import netCDF4 as nc
import matplotlib.pyplot as plt
import peak_detection
from kneed import KneeLocator
from read_chroma import read_chromato_and_chromato_cube
# from peak_detection import peak_detection
import peak_detection
from baseline_correction import chromato_reduced_noise
import skimage

In [ ]:
file_path = '/home/camille/Documents/app/data/input/A-F-028-817822-droite-ReCIVA.cdf'
# estimation with filtering_factor_estimation.ipynb
mod_time = 1.7
filtering_factor = 0.4

In [ ]:
tic_chromato, time_rn, spectra_obj = read_chroma.read_chroma(file_path, 1.7, None)


In [ ]:
from scipy.signal import savgol_filter
import pybaselines

def chromato_reduced_noise(chromato): 
    tmp = np.empty_like(chromato)
    for i in range(tmp.shape[1]):
        tmp[:, i] = savgol_filter(
           chromato[:, i] - pybaselines.whittaker.asls(chromato[:, i],
                                                       lam=1000.0, # 1000
                                                       p=0.1)[0], #0.05
           window_length=5,  # 5, 11 pour un lissage + fort
           polyorder=2,
           mode='nearest')
    tmp[tmp < .0] = 0
    return tmp

# def chromato_reduced_noise(chromato, savgol_window=5, lam=3000.0, p=0.05):
#     """Version améliorée avec meilleure réduction du bruit."""
#     tmp = np.empty_like(chromato)
#     for i in range(tmp.shape[1]):
#         # Correction de ligne de base
#         baseline_corrected = chromato[:, i] - pybaselines.whittaker.asls(
#             chromato[:, i], lam=lam, p=p)[0]
        
#         # Premier lissage Savitzky-Golay
#         smoothed = savgol_filter(baseline_corrected, savgol_window, 2, mode='nearest')
        
#         # Utiliser une moyenne mobile pour réduire davantage le bruit
#         # (peut être remplacée par un second Savitzky-Golay avec différents paramètres)
#         from scipy.ndimage import uniform_filter1d
#         smoothed = uniform_filter1d(smoothed, size=5)
        
#         # Seuillage pour supprimer le bruit de fond
#         noise_threshold = np.std(smoothed) * 0.1  # Ajuster selon votre cas
#         smoothed[smoothed < noise_threshold] = 0
        
#         tmp[:, i] = smoothed
    
#     # Élimination des valeurs négatives
#     tmp[tmp < 0.0] = 0
    
#     return tmp

# def wavelet_denoising(chromato):
#     """Débruitage par ondelettes."""
#     from pywt import wavedec, waverec, threshold
    
#     denoised = np.empty_like(chromato)
#     for i in range(chromato.shape[1]):
#         # Décomposition en ondelettes
#         coeffs = wavedec(chromato[:, i], 'sym8', level=3)
        
#         # Seuillage des coefficients
#         for j in range(1, len(coeffs)):
#             coeffs[j] = threshold(coeffs[j], value=np.std(coeffs[j])*0.8, mode='soft')
        
#         # Reconstruction
#         denoised[:, i] = waverec(coeffs, 'sym8')
    
#     denoised[denoised < 0] = 0
#     return denoised


chromato_tic_preprocessed = chromato_reduced_noise(tic_chromato)
# chromato_tic_preprocessed = wavelet_denoising(tic_chromato)


In [ ]:
filtering_factor = 50 #TODO
sigma = skimage.restoration.estimate_sigma(chromato_tic_preprocessed, channel_axis=None)
# chromato_tic, time_rn, chromato_cube, sigma, mass_range = read_chromato_and_chromato_cube(file_path, mod_time, pre_process=True)
dynamic_threshold_fact = filtering_factor * sigma * 100 / np.max(chromato_tic_preprocessed)
dynamic_threshold_fact

In [ ]:
plot.visualizer((tic_chromato, time_rn), title="chromato", log_chromato=False,mod_time=1.70)

In [ ]:
peaks_coordinates = peak_detection.peak_detection((chromato_tic_preprocessed, time_rn, None), spectra=None, chromato_cube=None, 
                                                  dynamic_threshold_fact=0.01,
                                                ABS_THRESHOLDS=None, 
                                                method="peak_local_max", 
                                                mode="tic", 
                                                cluster=False, 
                                                unique=False)
len(peaks_coordinates)

In [ ]:
coordinates_in_chromato = projection.matrix_to_chromato(peaks_coordinates, time_rn, mod_time, chromato_tic_preprocessed.shape)
plot.visualizer((chromato_tic_preprocessed, time_rn), title="chromato", log_chromato=False, points=coordinates_in_chromato, mod_time=1.70)  

In [ ]:
# determiner eps avec k-distance
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# Prétraitement : normalisation des données car DBSCAN est sensible à l'échelle, et RT1 et RT2 sont sur des échelles différentes
# En normalisant, les deux dimensions ont une moyenne 0 et un écart-type 1, donc elles sont sur la même échelle, ce qui évite un biais de distance.
data_scaled = StandardScaler().fit_transform(peaks_coordinates[:, :2]) 
# data_scaled = peaks_coordinates[:, :2]

# Paramètre k (= minPts pour DBSCAN)
# k = 4  # Valeur adaptée pour GC×GC-MSi
k = 4

# Calcul des k-distances
# instancie au k neighbors
neighbors = NearestNeighbors(n_neighbors=k)
# entraine les data, creer une structure de recherche du voisin
neighbors_fit = neighbors.fit(data_scaled)
# aplique la recherche des voisins a chaque point
distances, indices = neighbors_fit.kneighbors(data_scaled)
# tableau contenant , pour chaque point, la distance de ces k voisins
k_distances = np.sort(distances[:, k-1])




In [ ]:
# Visualisation pour identifier le coude
plt.figure(figsize=(10, 6))
plt.plot(range(len(k_distances)), k_distances)
plt.xlabel('Points triés par k-distance')
plt.ylabel(f'Distance au {k}-ième plus proche voisin')
plt.title('Courbe des k-distances')
plt.grid(True)

# Ajouter des lignes horizontales pour faciliter la sélection
percentiles = [50, 75, 90, 95]
for p in percentiles:
    d = np.percentile(k_distances, p)
    plt.axhline(y=d, color='r', linestyle='--', alpha=0.5)
    plt.text(len(k_distances)*0.02, d*1.02, f'{p}th percentile: {d:.4f}')

# Zoom
plt.figure(figsize=(10, 6))
plt.plot(range(len(k_distances))[200:], k_distances[200:])
plt.xlabel('Points triés par k-distance (zoom)')
plt.ylabel(f'Distance au {k}-ième plus proche voisin')
plt.title('Identification du point d\'inflexion pour eps')
plt.grid(True)

 # Trouver automatiquement le "coude" dans la courbe
knee = KneeLocator(
        range(len(k_distances)), 
        k_distances, 
        S=1.0, 
        curve="convex", 
        direction="increasing"
    )


optimal_eps = k_distances[knee.knee]
print(f"Valeur optimale pour eps : {optimal_eps}")
plt.axhline(y=optimal_eps, color='r', linestyle='--', 
                label=f'optimal eps = {optimal_eps:.4f}')
plt.legend()
plt.show()



Interprétation du graphique :

Les points avant le coude correspondent généralement aux pics bien définis
Les points après le coude peuvent correspondre à du bruit ou des pics très isolés

In [ ]:
# utilisation Dbscan avec optimal eps
from sklearn.cluster import DBSCAN


# Application de DBSCAN avec les paramètres choisis
dbscan = DBSCAN(eps=optimal_eps, min_samples=k).fit(data_scaled)
# attribu  un entier a 1 cluster et  -1 a un bruit, chaque point se voit attribuer le label du cluster auquel il appartient
labels = dbscan.labels_


# Statistiques basiques sur le clustering
# compte le nombre d'étiquettes uniques et ajuste ce compte en soustrayant 1 si le bruit est présent => obtenir le nombre de clusters valides.
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = list(labels).count(-1)
print(f"Nombre de clusters: {n_clusters}")
print(f"Nombre de points considérés comme bruit: {n_noise} ({n_noise/len(labels)*100:.1f}%)")

# Visualisation des clusters sur le plan RT1×RT2
plt.figure(figsize=(12, 8))
# Pour les points de bruit, recreer l array avec des booleens
noise_mask = labels == -1
plt.scatter(data_scaled[noise_mask, 0], data_scaled[noise_mask, 1], color='gray', alpha=0.3, s=10, label='Bruit')
# Pour les clusters
unique_labels = set(labels)
unique_labels.discard(-1)
for label in unique_labels:
    cluster_mask = labels == label
    plt.scatter(data_scaled[cluster_mask, 0], data_scaled[cluster_mask, 1], alpha=0.7, s=30, label=f'Cluster {label}')

plt.xlabel('RT1')
plt.ylabel('RT2')
plt.title(f'Résultats DBSCAN avec eps={optimal_eps}, minPts={k}')
plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
# estimation du eps clusters/ bruit
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import numpy as np

min_pts = 4
eps_values = np.linspace(0.1, 0.5, 10)  # 10 valeurs de eps
fig, axes = plt.subplots(2, len(eps_values) // 2, figsize=(16, 10))
axes = axes.flatten()

for i, eps in enumerate(eps_values):
    clusters = DBSCAN(eps=eps, min_samples=min_pts).fit_predict(data_scaled)  # fit + labels

    # Nombre de clusters et points de bruit
    n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
    n_noise = list(clusters).count(-1)

    ax = axes[i]
    noise_mask = clusters == -1

    # Bruit
    ax.scatter(data_scaled[noise_mask, 0], data_scaled[noise_mask, 1],
               color='gray', alpha=0.3, s=10, label='Bruit')

    # Clusters
    unique_labels = set(clusters)
    unique_labels.discard(-1)
    for label in unique_labels:
        cluster_mask = clusters == label
        ax.scatter(data_scaled[cluster_mask, 0], data_scaled[cluster_mask, 1],
                   alpha=0.7, s=30, label=f'Cluster {label}')

    ax.set_xlabel('RT1')
    ax.set_ylabel('RT2')
    ax.set_title(f'eps:{eps:.3f}, clusters:{n_clusters}, bruit:{n_noise}')
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# estimation du eps clusters/ bruit
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import numpy as np

min_pts = 4
eps_values = np.linspace(0.15, 0.25, 10)  # 10 valeurs de eps
fig, axes = plt.subplots(2, len(eps_values) // 2, figsize=(16, 10))
axes = axes.flatten()

for i, eps in enumerate(eps_values):
    clusters = DBSCAN(eps=eps, min_samples=min_pts).fit_predict(data_scaled)  # fit + labels

    # Nombre de clusters et points de bruit
    n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
    n_noise = list(clusters).count(-1)

    ax = axes[i]
    noise_mask = clusters == -1

    # Bruit
    ax.scatter(data_scaled[noise_mask, 0], data_scaled[noise_mask, 1],
               color='gray', alpha=0.3, s=10, label='Bruit')

    # Clusters
    unique_labels = set(clusters)
    unique_labels.discard(-1)
    for label in unique_labels:
        cluster_mask = clusters == label
        ax.scatter(data_scaled[cluster_mask, 0], data_scaled[cluster_mask, 1],
                   alpha=0.7, s=30, label=f'Cluster {label}')

    ax.set_xlabel('RT1')
    ax.set_ylabel('RT2')
    ax.set_title(f'eps:{eps:.3f}, clusters:{n_clusters}, bruit:{n_noise}')
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# utilisation Dbscan avec eps =0.161
from sklearn.cluster import DBSCAN


# Application de DBSCAN avec les paramètres choisis
dbscan = DBSCAN(eps=0.161, min_samples=k).fit(data_scaled)
# attribu  un entier a 1 cluster et  -1 a un bruit, chaque point se voit attribuer le label du cluster auquel il appartient
labels = dbscan.labels_


# Statistiques basiques sur le clustering
# compte le nombre d'étiquettes uniques et ajuste ce compte en soustrayant 1 si le bruit est présent => obtenir le nombre de clusters valides.
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = list(labels).count(-1)
print(f"Nombre de clusters: {n_clusters}")
print(f"Nombre de points considérés comme bruit: {n_noise} ({n_noise/len(labels)*100:.1f}%)")

# Visualisation des clusters sur le plan RT1×RT2
plt.figure(figsize=(12, 8))
# Pour les points de bruit, recreer l array avec des booleens
noise_mask = labels == -1
plt.scatter(data_scaled[noise_mask, 0], data_scaled[noise_mask, 1], color='gray', alpha=0.3, s=10, label='Bruit')
# Pour les clusters
unique_labels = set(labels)
unique_labels.discard(-1)
for label in unique_labels:
    cluster_mask = labels == label
    plt.scatter(data_scaled[cluster_mask, 0], data_scaled[cluster_mask, 1], alpha=0.7, s=30, label=f'Cluster {label}')

plt.xlabel('RT1')
plt.ylabel('RT2')
plt.title(f'Résultats DBSCAN avec eps={0.161}, minPts={k}')
# plt.legend(loc='best')
plt.grid(True)
plt.show()

Évaluation :
Vérifier si les pics connus d'un même composé sont regroupés correctement
Comparez le nombre de clusters et leur cohérence chimique

#indice de silhouette simplifiee


# a(i) : la distance moyenne du point à tous les autres points de son propre cluster (cohésion) (intracluster)
# b(i) : la distance moyenne minimale du point aux points du cluster voisin le plus proche (séparation)(extra cluster)

# L'indice de silhouette s(i) = (b(i) - a(i)) / max(a(i), b(i))

In [ ]:
# verifier la qualite avec l indice de silhouette
from sklearn.metrics import silhouette_score, calinski_harabasz_score

def simplified_silhouette_for_dbscan(X, labels):
    """Calcule un indice de silhouette simplifié adapté à DBSCAN"""
    unique_labels = set(labels)
    unique_labels.discard(-1)
    
    #  Si moins de 2 clusters, pas de sens de calculer la silhouette
    if len(unique_labels) < 2:
        return np.nan
    
    # Points qui ne sont pas du bruit
    valid_points = labels != -1
    X_valid = X[valid_points]
    labels_valid = labels[valid_points]
    
    # Utiliser la fonction standard de sklearn si possible
    try:
        return silhouette_score(X_valid, labels_valid)
    except:
        # Version simplifiée (si trop de clusters ou autres problèmes)
        silhouette_scores = []
        cluster_centers = {}
        
        # Calculer les centroïdes
        for label in unique_labels:
            cluster_points = X[labels == label]
            cluster_centers[label] = np.mean(cluster_points, axis=0)
        
        # Pour chaque point non-bruit
        for i, (point, label) in enumerate(zip(X, labels)):
            if label == -1:
                continue
                
            # a(i): Distance au centroïde de son propre cluster
            a_i = np.linalg.norm(point - cluster_centers[label])
            
            # b(i): Distance au centroïde du cluster le plus proche
            b_i = np.inf
            for other_label in unique_labels:
                if other_label != label:
                    dist = np.linalg.norm(point - cluster_centers[other_label])
                    b_i = min(b_i, dist)
            
            # Score de silhouette
            if max(a_i, b_i) > 0:
                silhouette_scores.append((b_i - a_i) / max(a_i, b_i))
        
          # Utiliser la médiane au lieu de la moyenne pour le score final
        return np.median(silhouette_scores) if silhouette_scores else np.nan
        # return np.mean(silhouette_scores) if silhouette_scores else np.nan

# Calcul des métriques de qualité
silhouette = simplified_silhouette_for_dbscan(data_scaled, labels)
print(f"Indice de silhouette simplifié: {silhouette:.4f}")

# # Calcul des distances intra-cluster
# intra_distances = []
# for label in set(labels):
#     if label != -1:
#         cluster_points = data_scaled[labels == label]
#         if len(cluster_points) > 1:
#             centroid = np.mean(cluster_points, axis=0)
#             dists = np.linalg.norm(cluster_points - centroid, axis=1)
#             intra_distances.append(np.mean(dists))

# avg_intra = np.mean(intra_distances) if intra_distances else np.nan
# print(f"Distance intra-cluster moyenne: {avg_intra:.4f}")

In [ ]:
# option : optimisation fine
# Tester plusieurs valeurs d'eps autour de celle identifiée
# eps_values = np.linspace(optimal_eps * 0.7, optimal_eps * 1.3, 7)  # ±30% autour de votre eps initial
eps_values = np.linspace(0.033 * 0.7, 0.033 * 1.3, 7)  # ±30% autour de votre eps initial
results = []

for eps in eps_values:
    dbscan = DBSCAN(eps=eps, min_samples=k).fit(data_scaled)
    labels = dbscan.labels_
    
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)
    silhouette = simplified_silhouette_for_dbscan(data_scaled, labels)
    
    # Calcul des distances intra-cluster
    intra_distances = []
    for label in set(labels):
        if label != -1:
            cluster_points = data_scaled[labels == label]
            if len(cluster_points) > 1:
                centroid = np.mean(cluster_points, axis=0)
                dists = np.linalg.norm(cluster_points - centroid, axis=1)
                intra_distances.append(np.median(dists))
    
    avg_intra = np.median(intra_distances) if intra_distances else np.nan
    
    results.append({
        'eps': eps,
        'n_clusters': n_clusters,
        'n_noise': n_noise,
        'silhouette': silhouette,
        'median_intra': avg_intra
    })

# Afficher les résultats sous forme de tableau
import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)

# Visualiser les métriques de qualité
fig, axs = plt.subplots(1, 2, figsize=(14, 6))
axs[0].plot(results_df['eps'], results_df['silhouette'], 'o-')
axs[0].set_title('Indice de silhouette')
axs[0].set_xlabel('eps')
axs[0].grid(True)

axs[1].plot(results_df['eps'], results_df['n_clusters'], 'o-', label='Clusters')
axs[1].plot(results_df['eps'], results_df['n_noise'], 'o--', label='Points de bruit')
axs[1].set_title('Nombre de clusters et points de bruit')
axs[1].set_xlabel('eps')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()

# Identifier la meilleure valeur d'eps selon la silhouette
best_idx = results_df['silhouette'].idxmax()
best_eps = results_df.loc[best_idx, 'eps']
print(f"Meilleure valeur d'eps selon la silhouette: {best_eps:.4f}")

In [ ]:
#Comparaison médiane vs moyenne (optionnel).
#  Comparer l'approche par moyenne vs médiane
results_comparative = []

for eps in [eps_choisi]:  # Ou testez plusieurs valeurs
    dbscan = DBSCAN(eps=eps, min_samples=k).fit(X_scaled)
    labels = dbscan.labels_
    
    # Calcul avec moyenne
    intra_mean = []
    for label in set(labels) - {-1}:
        cluster_points = X_scaled[labels == label]
        if len(cluster_points) > 1:
            centroid = np.mean(cluster_points, axis=0)
            dists = np.linalg.norm(cluster_points - centroid, axis=1)
            intra_mean.append(np.mean(dists))
    avg_intra_mean = np.mean(intra_mean) if intra_mean else np.nan
    
    # Calcul avec médiane
    intra_median = []
    for label in set(labels) - {-1}:
        cluster_points = X_scaled[labels == label]
        if len(cluster_points) > 1:
            centroid = np.mean(cluster_points, axis=0)
            dists = np.linalg.norm(cluster_points - centroid, axis=1)
            intra_median.append(np.median(dists))
    avg_intra_median = np.median(intra_median) if intra_median else np.nan
    
    # Identifier les clusters potentiellement problématiques
    problematic_clusters = []
    for label in set(labels) - {-1}:
        cluster_points = X_scaled[labels == label]
        if len(cluster_points) > 1:
            centroid = np.mean(cluster_points, axis=0)
            dists = np.linalg.norm(cluster_points - centroid, axis=1)
            mean_dist = np.mean(dists)
            median_dist = np.median(dists)
            # Si grande différence entre moyenne et médiane
            if mean_dist > median_dist * 1.5:
                problematic_clusters.append(label)
    
    results_comparative.append({
        'eps': eps,
        'mean_metric': avg_intra_mean,
        'median_metric': avg_intra_median,
        'problematic_clusters': problematic_clusters,
        'n_problematic': len(problematic_clusters)
    })

print("Comparaison moyenne vs médiane:")
print(pd.DataFrame(results_comparative))
if results_comparative[0]['n_problematic'] > 0:
    print(f"Clusters à examiner (potentiellement avec outliers): {results_comparative[0]['problematic_clusters']}")

Avantages de l'utilisation de la médiane pour GC×GC-MS

Résistance aux valeurs aberrantes : Les données GC×GC-MS contiennent souvent des pics aberrants ou mal définis qui peuvent fausser les moyennes.
Meilleure représentation des pics typiques : La médiane reflète mieux la taille et forme "typique" des pics chromatographiques.
Stabilité accrue : Les résultats seront moins sensibles aux variations mineures dans la préparation des échantillons ou aux artefacts d'acquisition.
Robustesse face au bruit : Dans les régions de haute densité du chromatogramme, certains points peuvent avoir des distances anormalement élevées sans être des outliers légitimes.

In [ ]:
#validation du choix d eps: calcul des distances intra_ cluster (evalu la compaciteé des clusters) : on veut des cluster compacts
from sklearn.metrics import pairwise_distances
labels = clustering.labels_
unique_labels = set(labels)
unique_labels.discard(-1)  # Ignore noise
intra_distances =[]
for label in unique_labels:
    cluster_points = coordinates_all_mass[labels == label]
    if len(cluster_points) > 1:
            centroid = np.mean(cluster_points, axis=0)
            dists = np.linalg.norm(cluster_points - centroid, axis=1)
            intra_distances.append(np.mean(dists))
            overall_mean = np.mean(intra_distances) if intra_distances else np.nan

            #: Plus la valeur est petite, plus les clusters sont compacts, ce qui est souvent souhaitable dans l'analyse GC×GC-MS car cela signifie que les points d'un même composé sont bien regroupés.
            # Dans l'analyse GC×GC-MS, des clusters compacts suggèrent souvent que le regroupement correspond bien à des composés chimiques réels.

In [ ]:
# distances inter cluste r: evaluer si les clusters sont bien séparés

In [ ]:
!pip install hdbscan

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=k)
labels = clusterer.fit_predict(data_scaled)

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = list(labels).count(-1)
print(f"Nombre de clusters: {n_clusters}")
print(f"Nombre de points considérés comme bruit: {n_noise} ({n_noise/len(labels)*100:.1f}%)")

# Visualisation des clusters sur le plan RT1×RT2
plt.figure(figsize=(12, 8))
# Pour les points de bruit, recreer l array avec des booleens
noise_mask = labels == -1
plt.scatter(data_scaled[noise_mask, 0], data_scaled[noise_mask, 1], color='gray', alpha=0.3, s=10, label='Bruit')
# Pour les clusters
unique_labels = set(labels)
unique_labels.discard(-1)
for label in unique_labels:
    cluster_mask = labels == label
    plt.scatter(data_scaled[cluster_mask, 0], data_scaled[cluster_mask, 1], alpha=0.7, s=30, label=f'Cluster {label}')

plt.xlabel('RT1')
plt.ylabel('RT2')
plt.title(f'Résultats HDBSCAN, minPts={k}')
# plt.legend(loc='best')
plt.grid(True)
plt.show()